In [ ]:
# Load the data
import numpy as np
import cv2
import matplotlib.pyplot as plt

# Load the CSV data into a 2D numpy array
csv_absolute_filename='/mnt/hgfs/share/input/le_capture_20241119_110900.csv'
data = np.loadtxt(csv_absolute_filename, delimiter=',', skiprows=1, usecols=range(2, 80), dtype=int)

# Truncate and rotate the data 90 degrees counterclockwise
data = data[0:1200,:]
data_rotated = np.rot90(data, 1)

print(f"Data shape: {data.shape}")

In [ ]:
# Show the data
plt.figure(figsize=(15, 6))
time_per_sample = 200e-6
frequency_per_band = 1e6

# Calculate the extent
num_time_samples = data.shape[0]
num_frequency_bands = data.shape[1]
extent = [0, num_time_samples * time_per_sample, 0, num_frequency_bands * frequency_per_band]

plt.imshow(data_rotated, cmap='hot', aspect='auto', interpolation='none', origin='lower', extent=extent)

print(f"The number of time samples is {num_time_samples} and the number of frequency bands is {num_frequency_bands}")
plt.colorbar(label='dB')
plt.title('Spectrogram of Microwave Power 50%')
plt.xlabel('Time (seconds)')
plt.ylabel('Frequency (Hz)')
plt.show()

In [ ]:
# Analyze frequency variations

# Calculate frequencies
num_frequency_bands = data_rotated.shape[0]
frequencies = np.arange(num_frequency_bands) * frequency_per_band

# Calculate times
num_time_samples = data_rotated.shape[1]
times = np.arange(num_time_samples) * time_per_sample

# Find the frequency index with the maximum power at each time sample
max_freq_indices = np.argmax(data_rotated, axis=0)

# Convert frequency indices to actual frequencies
max_frequencies = max_freq_indices * frequency_per_band

# Calculate weighted average frequency at each time sample
# Ensure data_rotated is of type float for accurate calculations
data_rotated_float = data_rotated.astype(float)

# Calculate the sum of power over frequencies at each time sample
power_sum = np.sum(data_rotated_float, axis=0)

# Avoid division by zero
power_sum[power_sum == 0] = np.finfo(float).eps

# Calculate the weighted sum of frequencies
weighted_freq_sum = np.sum(data_rotated_float * frequencies[:, np.newaxis], axis=0)

# Calculate the weighted average frequency
average_frequencies = weighted_freq_sum / power_sum

# Calculate the variance and standard deviation (bandwidth) of frequencies at each time sample
variance_frequencies = np.sum(data_rotated_float * (frequencies[:, np.newaxis] - average_frequencies) ** 2, axis=0) / power_sum
std_frequencies = np.sqrt(variance_frequencies)

# Plot frequency with maximum power over time
plt.figure(figsize=(15, 6))
plt.plot(times, max_frequencies, label='Max Power Frequency', alpha=0.7)
plt.plot(times, average_frequencies, label='Average Frequency', alpha=0.7)
plt.title('Frequency Analysis Over Time')
plt.xlabel('Time (seconds)')
plt.ylabel('Frequency (Hz)')
plt.legend()
plt.show()

# # Plot bandwidth over time
# plt.figure(figsize=(15, 6))
# plt.plot(times, std_frequencies, label='Bandwidth (Std Dev of Frequencies)')
# plt.title('Bandwidth Over Time')
# plt.xlabel('Time (seconds)')
# plt.ylabel('Frequency (Hz)')
# plt.legend()
# plt.show()

In [ ]:

# Overlay the bandwidth on the spectrogram
plt.figure(figsize=(15, 6))

# Plot the spectrogram
plt.imshow(data_rotated, cmap='hot', aspect='auto', interpolation='none', origin='lower', extent=extent)

std_frequencies_mean=np.mean(std_frequencies)

# Overlay the average frequency
#plt.plot(times, average_frequencies, 'w-', linewidth=2, label='Average Frequency')

# Fill between average_frequency - std_frequency and average_frequency + std_frequency
# plt.fill_between(times, average_frequencies - std_frequencies, average_frequencies + std_frequencies,
#                  color='white', alpha=0.3, label='Bandwidth')
plt.plot(times, np.abs(std_frequencies - std_frequencies_mean)*23+std_frequencies_mean, 'g-', linewidth=2, label='Pseudo Bandwidth/Power Estimate')
plt.colorbar(label='dB')
plt.title('Spectrogram with Bandwidth Over Time')
plt.xlabel('Time (seconds)')
plt.ylabel('Frequency (Hz)')
plt.legend()
plt.show()

In [ ]:
# Load the data
import numpy as np
import cv2
import matplotlib.pyplot as plt
from skimage.feature import peak_local_max

# Load the CSV data into a 2D numpy array
csv_absolute_filename = '/mnt/hgfs/share/input/le_capture.csv'
data = np.loadtxt(csv_absolute_filename, delimiter=',', skiprows=1, usecols=range(2, 80), dtype=int)

# Truncate and rotate the data 90 degrees counterclockwise
data = data[0:800, :]
data_rotated = np.rot90(data, 1)

print(f"Data shape: {data.shape}")

# Show the data
plt.figure(figsize=(15, 6))
time_per_sample = 200e-6
frequency_per_band = 1e6

# Calculate the extent
num_time_samples = data.shape[0]
num_frequency_bands = data.shape[1]
extent = [0, num_time_samples * time_per_sample, 0, num_frequency_bands * frequency_per_band]

# plt.imshow(data_rotated, cmap='hot', aspect='auto', interpolation='none', origin='lower', extent=extent)

# print(f"The number of time samples is {num_time_samples} and the number of frequency bands is {num_frequency_bands}")
# plt.colorbar(label='dB')
# plt.title('Spectrogram of RF Power 50%')
# plt.xlabel('Time (seconds)')
# plt.ylabel('Frequency (Hz)')
# plt.show()

# -------------------- Added Code to Detect and Plot Outliers --------------------

# Analyze the data to find outliers

# Ensure the data is in float format for accurate processing
data_rotated_float = data_rotated.astype(float)

# Normalize the data
data_normalized = (data_rotated_float - np.min(data_rotated_float)) / (np.max(data_rotated_float) - np.min(data_rotated_float))

# Use peak_local_max to find peaks (outliers) in the data
coordinates = peak_local_max(data_normalized, min_distance=5, threshold_rel=0.5)

# Extract frequency and time indices of the peaks
freq_indices = coordinates[:, 0]
time_indices = coordinates[:, 1]

# Convert indices to actual time and frequency values
times_peaks = time_indices * time_per_sample
frequencies_peaks = freq_indices * frequency_per_band

# Plot the spectrogram with outliers highlighted
plt.figure(figsize=(15, 6))
plt.imshow(data_rotated, cmap='hot', aspect='auto', interpolation='none', origin='lower', extent=extent)

# Overlay the detected peaks on the spectrogram
plt.scatter(times_peaks, frequencies_peaks, s=50, facecolors='green', edgecolors='green', label='Detected Outliers')

plt.colorbar(label='dB')
plt.title('Spectrogram with Detected Outliers')
plt.xlabel('Time (seconds)')
plt.ylabel('Frequency (Hz)')
plt.legend()
plt.show()
